In [1]:
# running in python 3.5
import numpy as np
import numba

In [2]:

@numba.jit(nopython = True)
def get_char_pol(A):
    """
    This uses Newton's identities to get the characteristic polynomial
    """
    assert(A.shape[0] == A.shape[1]) # it must be a square matrix
    n = A.shape[0] 
    charpoly_array = np.zeros(n + 1)
    traces_array = np.zeros(n + 1)
    
    traces_array[0] = n
    working_matrix = A.copy()
    
    traces_array[1] = np.trace(working_matrix)
    tracer_idx = 1
    # note trace (A^1) will be in the position 1...    
    # a_0, a_1, a_2, ..., a_n
    charpoly_array[n] = 1
    
    for r in range(1, n):
        # through n-1, then separate handling at the end for a_0
        a_idx = n - r
        running_sum = 0        
        
        working_matrix = working_matrix @ A
        tracer_idx += 1
        traces_array[tracer_idx] = np.trace(working_matrix)
        for j in range(1, r + 1):
            running_sum += charpoly_array[a_idx + j] * traces_array[j]
            # this itself can go in a memo I think... simplisitically with np.zeros and such... 
            # slightly more complicated with a buffer / pivot to which we don't go past
        charpoly_array[a_idx] += - running_sum / r
    
    another_running_sum = 0
    for i in range(1, n + 1):
        another_running_sum += charpoly_array[i] * traces_array[i]
    charpoly_array[0] = -1/n * another_running_sum 
    return charpoly_array
        
        
        
    


In [3]:
m = 10
# A = np.random.rand(m*m)
A = np.random.randint(-50, 10,m*m)

A = np.float64(np.reshape(A, (m,m))) * 10
# numba gets upset at typing issues if A is not in floats...

result = get_char_pol(A)
result

# in general the idea is elegant, though there seems to be floating point issues, with say, n = 20

array([ -2.30533239e+25,  -6.90016976e+23,  -9.23285658e+20,
         3.38543128e+17,  -4.27940077e+16,  -8.30442153e+13,
         6.70472600e+10,  -7.68550000e+07,   4.54600000e+05,
         2.32000000e+03,   1.00000000e+00])

In [4]:
eigvals = np.linalg.eigvals(A)
polynomials = np.poly(eigvals)
# c[0] * x**(N) + c[1] * x**(N-1) + ... + c[N-1] * x + c[N]
# note that these show up in reversed order...
reversed_polynomial_array = polynomials[::-1]
reversed_polynomial_array


array([ -2.30533239e+25,  -6.90016976e+23,  -9.23285658e+20,
         3.38543128e+17,  -4.27940077e+16,  -8.30442153e+13,
         6.70472600e+10,  -7.68550000e+07,   4.54600000e+05,
         2.32000000e+03,   1.00000000e+00])

In [5]:
np.linalg.det(A)

-2.3053323881371129e+25